In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.ensemble import BaggingRegressor

# Đọc dữ liệu từ CSV vào DataFrame
data = pd.read_csv('/kaggle/input/throughput-prediction/datasetNov1_data_full.csv')

In [17]:
# Chia input và output
X = data.iloc[:, 0:11].values
y = data.iloc[:, 11:13].values
print(X)
print(y)

[[5.0e+00 2.0e+00 1.0e+00 ... 3.4e+02 5.9e+02 1.0e+00]
 [3.0e+00 5.0e+00 6.0e+00 ... 3.8e+02 5.5e+02 1.5e+00]
 [5.0e+00 5.0e+00 4.0e+00 ... 3.5e+02 5.7e+02 5.0e-01]
 ...
 [4.0e+00 4.0e+00 3.0e+00 ... 3.2e+02 5.0e+02 1.5e+00]
 [6.0e+00 3.0e+00 4.0e+00 ... 4.0e+02 5.3e+02 5.0e-01]
 [1.0e+00 4.0e+00 3.0e+00 ... 3.7e+02 5.0e+02 1.5e+00]]
[[0.00000000e+00 6.22222222e-01]
 [1.11239380e-02 8.57025920e-01]
 [0.00000000e+00 3.05555556e-01]
 ...
 [9.76509144e-01 2.27078280e-02]
 [3.01000000e-05 2.77769408e-01]
 [1.55790585e-01 7.45718317e-01]]


In [18]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)




In [19]:
# Xây dựng mô hình mạng học sâu
# Thay đổi kiến trúc mạng
model = Sequential([
    Dense(512, activation='relu', input_shape=(11,)),
   # Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2),
])

model.summary()
# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               6144      
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 32)                4128      
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                                 
 dense_17 (Dense)            (None, 2)                 34        
                                                                 
Total params: 93010 (363.32 KB)
Trainable params: 9301

In [20]:
# Compile the model with learning rate schedule
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)

model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mean_squared_error')


In [21]:
# Chia dữ liệu thành tập huấn luyện và tập xác nhận
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)


In [22]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=300, batch_size=128, validation_data=(X_val, y_val), verbose=1)

Epoch 1/300
651/651 [==============================] - 5s 6ms/step - loss: 0.0224 - val_loss: 0.0046
Epoch 2/300
651/651 [==============================] - 4s 6ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 3/300
651/651 [==============================] - 4s 6ms/step - loss: 0.0011 - val_loss: 9.0566e-04
Epoch 4/300
651/651 [==============================] - 4s 6ms/step - loss: 9.5033e-04 - val_loss: 7.5385e-04
Epoch 5/300
651/651 [==============================] - 4s 6ms/step - loss: 7.5276e-04 - val_loss: 6.0933e-04
Epoch 6/300
651/651 [==============================] - 4s 6ms/step - loss: 8.3712e-04 - val_loss: 0.0011
Epoch 7/300
651/651 [==============================] - 4s 6ms/step - loss: 6.0106e-04 - val_loss: 8.9194e-04
Epoch 8/300
651/651 [==============================] - 4s 6ms/step - loss: 5.6473e-04 - val_loss: 4.1073e-04
Epoch 9/300
651/651 [==============================] - 4s 6ms/step - loss: 5.1106e-04 - val_loss: 2.5005e-04
Epoch 10/300
651/651 [=====================

In [23]:
# Đánh giá hiệu năng trên tập validation bằng RMSE
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("MSE:", mse)
print("RMSE trên tập validation:", rmse)

2602/2602 [==============================] - 4s 2ms/step
MSE: 1.1905164272169732e-05
RMSE trên tập validation: 0.0034503861047960604
